In [1]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [2]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [3]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [4]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [5]:
barbMetrics = loadMetricData('barbComp', "metrics")
barbMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-02 23:55:00,70,0,1.2,0.000055,84.2,81.5
1436,2016-05-02 23:56:00,73,0,1.2,0.000055,84.2,81.5
1437,2016-05-02 23:57:00,74,0,1.2,0.000057,84.2,81.5
1438,2016-05-02 23:58:00,74,0,1.2,0.000054,84.2,81.5
1439,2016-05-02 23:59:00,73,0,1.1,0.000053,83.3,80.6


In [6]:
barbMetrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [7]:
# clean metric data

barbMetrics.timestamp = pd.to_datetime(barbMetrics.timestamp)

barbMetrics.heartrate = barbMetrics.heartrate.fillna(0)
barbMetrics.steps = barbMetrics.steps.fillna(0)
barbMetrics.calories = barbMetrics.calories.fillna(0)
barbMetrics.gsr = barbMetrics.gsr.fillna(0)
barbMetrics.skintemp = barbMetrics.skintemp.fillna(0)
barbMetrics.airtemp = barbMetrics.airtemp.fillna(0)

barbMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-04-26 00:00:00,0,0,1.3,0,0,0
1,2016-04-26 00:01:00,0,0,1.1,0,0,0
2,2016-04-26 00:02:00,0,0,1.2,0,0,0
3,2016-04-26 00:03:00,0,0,1.1,0,0,0
4,2016-04-26 00:04:00,0,0,1.2,0,0,0


In [8]:
barbActivities = loadMetricData('barbComp', "activities")
barbActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-02 20:22:21,2016-05-02T20:22:21Z,America/New_York,-240,2016-05-02 20:30:10,2016-05-02T20:30:10Z,America/New_York,-240,walk,248,475,35.900002,7.816667,128.032257,NaN,NaN,complete,1,walk__2016-05-02T202221
1,2016-05-02 23:45:44,2016-05-02T23:45:44Z,America/New_York,-240,2016-05-02 23:52:29,2016-05-02T23:52:29Z,America/New_York,-240,walk,311,632,35.400002,6.750000,120.395256,NaN,NaN,complete,1,walk__2016-05-02T234544
2,2016-05-03 00:57:18,2016-05-03T00:57:18Z,America/New_York,-240,2016-05-03 01:00:19,2016-05-03T01:00:19Z,America/New_York,-240,walk,181,357,18.200001,3.016667,113.889503,NaN,NaN,complete,1,walk__2016-05-03T005718
3,2016-05-03 01:24:39,2016-05-03T01:24:39Z,America/New_York,-240,2016-05-03 01:30:02,2016-05-03T01:30:02Z,America/New_York,-240,walk,188,348,21.500000,5.383333,119.536720,NaN,NaN,complete,1,walk__2016-05-03T012439
4,2016-05-03 01:46:18,2016-05-03T01:46:18Z,America/New_York,-240,2016-05-03 01:55:22,2016-05-03T01:55:22Z,America/New_York,-240,walk,316,518,23.000000,9.066667,109.281647,NaN,NaN,complete,1,walk__2016-05-03T014618


In [9]:
barbSleep = loadMetricData('barbComp', "sleep")
barbSleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-03 06:42:37,2016-05-03T06:42:37Z,America/New_York,-240,2016-05-03 07:39:37,2016-05-03T07:39:37Z,America/New_York,-240,56,0,...,0,sleep,3420,65.300003,59.01646,NaN,NaN,complete,2,sleep__2016-05-03T064237


In [10]:
barbSleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset      float64
end time                object
end time ISO            object
end time timezone       object
end time offset        float64
light mins             float64
deep mins              float64
rem mins               float64
interruption mins      float64
unknown mins           float64
interruptions          float64
toss turns             float64
type                    object
actual seconds         float64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                float64
id                      object
dtype: object

In [11]:
barbJournal = loadMetricData('barbComp', "journal")
barbJournal.tail(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
113,5/2/16 7:15,5/2/16 8:15,5/2/16,7:15 AM,8:15 AM,Train Commute,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN
114,5/2/16 8:15,5/2/16 9:30,5/2/16,8:15 AM,9:30 AM,setting up computers,work,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN
115,5/2/16 9:30,5/2/16 11:13,5/2/16,9:30 AM,11:13 AM,Class and setting up computers,work,NaN,NaN,NaN,NaN,NaN,6,6,NaN,NaN,NaN,NaN
116,5/2/16 11:13,5/2/16 12:00,5/2/16,11:13 AM,12:00 PM,Work,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,5/2/16 12:00,5/2/16 13:18,5/2/16,12:00 PM,1:18 PM,meeting with anne,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
barbJournal.dtypes

starttime       object
endtime         object
date            object
start           object
end             object
activity        object
location        object
generalemo      object
qualitative    float64
excited        float64
happy          float64
calm           float64
anxious        float64
sad            float64
angry          float64
hungry         float64
tired          float64
bored          float64
dtype: object

In [13]:
# data= barbJournal.iloc[:0]
# clean metric data
barbJournal["starttime"]= pd.to_datetime(barbJournal.starttime)

# data= barbJournal.iloc[:1]
barbJournal["endtime"]= pd.to_datetime(barbJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

barbJournal.qualitative = barbJournal.qualitative.fillna(0)
barbJournal.excited = barbJournal.excited.fillna(0)
barbJournal.happy = barbJournal.happy.fillna(0)
barbJournal.calm = barbJournal.calm.fillna(0)
barbJournal.anxious = barbJournal.anxious.fillna(0)
barbJournal.sad = barbJournal.sad.fillna(0)
barbJournal.angry = barbJournal.angry.fillna(0)
barbJournal.hungry = barbJournal.hungry.fillna(0)
barbJournal.tired = barbJournal.tired.fillna(0)
barbJournal.bored = barbJournal.bored.fillna(0)

print(barbJournal.shape)
barbJournal.head(5)
# barbJournal.tail(5)

(118, 18)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,2,6,0,0,0,0,2,1
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,2,6,0,0,0,0,2,0
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,0,6,0,0,0,0,4,0
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,0,6,0,0,0,0,4,0
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,0,6,0,0,0,0,4,0


In [14]:

barbJournal['boolexcited'] =  barbJournal['excited'] > 0
barbJournal['boolhappy'] =  barbJournal['happy'] > 0
barbJournal['boolcalm'] =  barbJournal['calm'] > 0
barbJournal['boolanxious'] =  barbJournal['anxious'] > 0
barbJournal['boolsad'] =  barbJournal['sad'] > 0
barbJournal['boolangry'] =  barbJournal['angry'] > 0
barbJournal['booltired'] =  barbJournal['tired'] > 0
barbJournal['boolhungry'] =  barbJournal['hungry'] > 0
barbJournal['boolbored'] =  barbJournal['bored'] > 0


for i in range(0, barbJournal.shape[0]):
#     barbJournal['intqual'] = int(barbJournal.iloc[i, 8])
    barbJournal['intexcited'] = int(barbJournal.iloc[i, 9])
    barbJournal['inthappy'] = int(barbJournal.iloc[i, 10]) 
    barbJournal['intcalm'] = int(barbJournal.iloc[i, 11])
    barbJournal['intanxious'] = int(barbJournal.iloc[i, 12])
    barbJournal['intsad'] = int(barbJournal.iloc[i, 13]) 
    barbJournal['intangry'] = int(barbJournal.iloc[i, 14]) 
    barbJournal['inttired'] = int(barbJournal.iloc[i, 15]) 
    barbJournal['inthungry'] = int(barbJournal.iloc[i, 16]) 
    barbJournal['intbored'] = int(barbJournal.iloc[i, 17]) 
    
barbJournal['exercise'] = barbJournal['activity'].str.contains('exercise')
barbJournal['code'] = barbJournal['activity'].str.contains('code')
barbJournal['anne'] = barbJournal['activity'].str.contains('anne')
barbJournal['will'] = barbJournal['activity'].str.contains('will')
barbJournal['mick'] = barbJournal['activity'].str.contains('mick')
barbJournal['geyer'] = barbJournal['activity'].str.contains('geyer')

barbJournal['home'] = barbJournal['location'].str.contains('home')
barbJournal['work'] = barbJournal['location'].str.contains("work")
barbJournal['school'] = barbJournal['location'].str.contains("school")




boolexcited = barbJournal.iloc[:, 18].values.reshape(barbJournal.shape[0], 1)
boolhappy = barbJournal.iloc[:, 19].values.reshape(barbJournal.shape[0], 1)
boolcalm = barbJournal.iloc[:, 20].values.reshape(barbJournal.shape[0], 1)
boolanxious = barbJournal.iloc[:, 21].values.reshape(barbJournal.shape[0], 1)
boolsad = barbJournal.iloc[:, 22].values.reshape(barbJournal.shape[0], 1)
boolangry = barbJournal.iloc[:, 23].values.reshape(barbJournal.shape[0], 1)
booltired = barbJournal.iloc[:, 24].values.reshape(barbJournal.shape[0], 1)
boolhungry = barbJournal.iloc[:, 25].values.reshape(barbJournal.shape[0], 1)
boolbored = barbJournal.iloc[:, 26].values.reshape(barbJournal.shape[0], 1)
intexcited = barbJournal.iloc[:, 27].values.reshape(barbJournal.shape[0], 1)
inthappy = barbJournal.iloc[:, 28].values.reshape(barbJournal.shape[0], 1)
intcalm = barbJournal.iloc[:, 29].values.reshape(barbJournal.shape[0], 1)
intanxious = barbJournal.iloc[:, 30].values.reshape(barbJournal.shape[0], 1)
intsad = barbJournal.iloc[:, 31].values.reshape(barbJournal.shape[0], 1)
intangry = barbJournal.iloc[:, 32].values.reshape(barbJournal.shape[0], 1)
inttired = barbJournal.iloc[:, 33].values.reshape(barbJournal.shape[0], 1)
inthungry = barbJournal.iloc[:, 34].values.reshape(barbJournal.shape[0], 1)
intbored = barbJournal.iloc[:, 35].values.reshape(barbJournal.shape[0], 1)
exercise = barbJournal.iloc[:, 36].values.reshape(barbJournal.shape[0], 1)
code = barbJournal.iloc[:, 37].values.reshape(barbJournal.shape[0], 1)
anne = barbJournal.iloc[:, 38].values.reshape(barbJournal.shape[0], 1)
will = barbJournal.iloc[:, 39].values.reshape(barbJournal.shape[0], 1)
mick = barbJournal.iloc[:, 40].values.reshape(barbJournal.shape[0], 1)
geyer = barbJournal.iloc[:, 41].values.reshape(barbJournal.shape[0], 1)
home = barbJournal.iloc[:, 42].values.reshape(barbJournal.shape[0], 1)
work = barbJournal.iloc[:, 43].values.reshape(barbJournal.shape[0], 1)
school = barbJournal.iloc[:, 44].values.reshape(barbJournal.shape[0], 1)

barbJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,intbored,exercise,code,anne,will,mick,geyer,home,work,school
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,...,0,False,False,False,False,False,False,False,True,False
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,...,0,False,False,False,False,False,False,False,True,False
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,...,0,False,False,False,False,False,False,False,True,False
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,...,0,False,False,False,False,True,False,False,True,False
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,...,0,False,False,False,False,False,False,False,False,True


In [15]:
# barbJournal.dtypes

In [16]:
# barbJournal.excited.convert_objects(convert_numeric=True)

In [17]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [18]:
# indicator for walking
totalData = makeIndicators('excitement', barbMetrics, barbJournal, 0, 1, 18)
totalData = makeIndicators('happy', barbMetrics, barbJournal, 0, 1, 19)
totalData = makeIndicators('calm', barbMetrics, barbJournal, 0, 1, 20)
totalData = makeIndicators('anxious', barbMetrics, barbJournal, 0, 1, 21)
totalData = makeIndicators('sad', barbMetrics, barbJournal, 0, 1, 22)
totalData = makeIndicators('angry', barbMetrics, barbJournal, 0, 1, 23)
totalData = makeIndicators('hungry', barbMetrics, barbJournal, 0, 1, 24)
totalData = makeIndicators('tired', barbMetrics, barbJournal, 0, 1, 25)
totalData = makeIndicators('bored', barbMetrics, barbJournal, 0, 1, 26)

totalData = makeIndicators('exercise', barbMetrics, barbJournal, 0, 1, 36)
totalData = makeIndicators('code', barbMetrics, barbJournal, 0, 1, 37)
totalData = makeIndicators('anne', barbMetrics, barbJournal, 0, 1, 38)
totalData = makeIndicators('will', barbMetrics, barbJournal, 0, 1, 39)
totalData = makeIndicators('mick', barbMetrics, barbJournal, 0, 1, 40)
totalData = makeIndicators('geyer', barbMetrics, barbJournal, 0, 1, 41)
totalData = makeIndicators('home', barbMetrics, barbJournal, 0, 1, 42)
totalData = makeIndicators('work', barbMetrics, barbJournal, 0, 1, 43)
totalData = makeIndicators('school', barbMetrics, barbJournal, 0, 1, 44)


In [19]:
print(totalData.describe())

         heartrate        steps     calories          gsr     skintemp  \
count  8640.000000  8640.000000  8640.000000  8640.000000  8640.000000   
mean      5.796065     0.834722     0.549975     0.000003     5.425810   
std      21.923344     6.034856     0.828393     0.000013    20.735248   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     0.000000   
75%       0.000000     0.000000     1.200000     0.000000     0.000000   
max     152.000000   127.000000    10.100000     0.000065    88.700000   

           airtemp   excitement      happy      calm   anxious    ...      \
count  8640.000000         8640       8640      8640      8640    ...       
mean      5.261435  0.000694444  0.0665509  0.515162  0.366551    ...       
std      20.108585    0.0263447   0.249257  0.499799   0.48189    ...       
min       0.000000       

In [20]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0       1.3    0         0        0   
1 2016-04-26 00:01:00          0      0       1.1    0         0        0   
2 2016-04-26 00:02:00          0      0       1.2    0         0        0   
3 2016-04-26 00:03:00          0      0       1.1    0         0        0   
4 2016-04-26 00:04:00          0      0       1.2    0         0        0   

  excitement  happy   calm  ...    bored exercise   code   anne   will   mick  \
0      False  False  False  ...    False    False  False  False  False  False   
1      False  False  False  ...    False    False  False  False  False  False   
2      False  False  False  ...    False    False  False  False  False  False   
3      False  False  False  ...    False    False  False  False  False  False   
4      False  False  False  ...    False    False  False  False  False  False   

   geyer   home   work school  
0  False  False  F

In [21]:
totalData.to_csv('totsarah.csv')

In [22]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,bored,exercise,code,anne,will,mick,geyer,home,work,school
0,2016-04-26 00:00:00,0,0,1.3,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-04-26 00:01:00,0,0,1.1,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-04-26 00:02:00,0,0,1.2,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-04-26 00:03:00,0,0,1.1,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-04-26 00:04:00,0,0,1.2,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(8640, 29)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0       1.3    0         0        0   
1 2016-04-26 00:01:00          0      0       1.1    0         0        0   
2 2016-04-26 00:02:00          0      0       1.2    0         0        0   
3 2016-04-26 00:03:00          0      0       1.1    0         0        0   
4 2016-04-26 00:04:00          0      0       1.2    0         0        0   

  excitement  happy   calm ...   will   mick  geyer   home   work school  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       1     4    0   0  
1       1     4

In [24]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
code = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
anne = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
will = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
mick = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
geyer = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
home = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
work = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
school = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 27].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 28].values.reshape(totalData.shape[0], 1)

In [25]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
        exercise, 
        code, 
        anne, 
        will, 
        mick, 
        geyer, 
        home, 
        work, 
        school,
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(8640, 18)

In [26]:
# define y
y = totalData.excitement.values
y.shape

(8640,)

In [27]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [28]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [29]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [30]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00082671957672 0.000385802469136
 Accuracy Rate   0.999173280423    0.999614197531
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00038580246913580245, 0.99961419753086422, 0.0, 1.0)

In [31]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set        Test Set    
------------------- ---------------- -----------------
         Error Rate 0.00082671957672 0.000385802469136
      Accuracy Rate   0.999173280423    0.999614197531
     True Positives              0.0               0.0
     True Negatives               --               1.0


(0.00038580246913580245, 0.99961419753086422, 0.0, 1.0)

In [32]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0209986772487 0.0204475308642
 Accuracy Rate  0.979001322751  0.979552469136
True Positives             0.0             0.0
True Negatives              --  0.979930528753


(0.020447530864197531, 0.97955246913580252, 0.0, 0.97993052875337705)

In [33]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00082671957672 0.000385802469136
 Accuracy Rate   0.999173280423    0.999614197531
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00038580246913580245, 0.99961419753086422, 0.0, 1.0)

In [34]:
# define y
y = totalData.happy.values
y.shape

(8640,)

In [35]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [36]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [37]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0749007936508 0.0733024691358
 Accuracy Rate  0.925099206349  0.926697530864
True Positives  0.630071599045  0.621794871795
True Negatives              --  0.946223316913


(0.073302469135802475,
 0.92669753086419748,
 0.62179487179487181,
 0.94622331691297212)

In [38]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0692791005291 0.0601851851852
      Accuracy Rate  0.930720899471  0.939814814815
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.060185185185185182, 0.93981481481481477, 0.0, 1.0)

In [39]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.196263227513 0.185185185185
 Accuracy Rate 0.803736772487 0.814814814815
True Positives 0.658711217184 0.634615384615
True Negatives             -- 0.826354679803


(0.18518518518518517,
 0.81481481481481488,
 0.63461538461538458,
 0.82635467980295563)

In [40]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0694444444444 0.0605709876543
 Accuracy Rate  0.930555555556  0.939429012346
True Positives             0.0             0.0
True Negatives              --  0.999589490969


(0.060570987654320986, 0.93942901234567899, 0.0, 0.99958949096880134)

In [41]:
# define y
y = totalData.calm.values
y.shape

(8640,)

In [42]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [43]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [44]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.304232804233 0.295910493827
 Accuracy Rate 0.695767195767 0.704089506173
True Positives  0.49166132136 0.512378094524
True Negatives             -- 0.907069102462


(0.2959104938271605,
 0.7040895061728395,
 0.51237809452363092,
 0.90706910246227168)

In [45]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.282738095238        0.28125
      Accuracy Rate 0.717261904762        0.71875
     True Positives 0.644644002566 0.642160540135
     True Negatives             -- 0.799841143765


(0.28125, 0.71875, 0.64216054013503376, 0.79984114376489279)

In [46]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.408399470899 0.410108024691
 Accuracy Rate 0.591600529101 0.589891975309
True Positives 0.910198845414 0.903225806452
True Negatives             -- 0.258141382049


(0.41010802469135804,
 0.58989197530864201,
 0.90322580645161288,
 0.25814138204924542)

In [47]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.309193121693 0.314043209877
 Accuracy Rate 0.690806878307 0.685956790123
True Positives 0.481077613855 0.479369842461
True Negatives             -- 0.904686258936


(0.31404320987654322,
 0.68595679012345678,
 0.47936984246061515,
 0.90468625893566323)

In [48]:
# define y
y = totalData.anxious.values
y.shape

(8640,)

In [49]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [50]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [51]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.317791005291 0.318287037037
 Accuracy Rate 0.682208994709 0.681712962963
True Positives 0.528172240037 0.521341463415
True Negatives             -- 0.779850746269


(0.31828703703703703,
 0.68171296296296302,
 0.52134146341463417,
 0.77985074626865669)

In [52]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.279265873016 0.288966049383
      Accuracy Rate 0.720734126984 0.711033950617
     True Positives 0.426935409986 0.417682926829
     True Negatives             -- 0.890547263682


(0.28896604938271603,
 0.71103395061728403,
 0.41768292682926828,
 0.89054726368159209)

In [53]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.395006613757 0.361882716049
 Accuracy Rate 0.604993386243 0.638117283951
True Positives 0.957398076042 0.960365853659
True Negatives             --  0.44092039801


(0.36188271604938271,
 0.63811728395061729,
 0.96036585365853655,
 0.44092039800995025)

In [54]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.349041005291 0.333333333333
 Accuracy Rate 0.650958994709 0.666666666667
True Positives 0.922125515346  0.92581300813
True Negatives             -- 0.508084577114


(0.33333333333333331,
 0.66666666666666674,
 0.92581300813008127,
 0.50808457711442789)

In [55]:
# define y
y = totalData.sad.values
y.shape

(8640,)

In [56]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [57]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [58]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.128141534392 0.125385802469
 Accuracy Rate 0.871858465608 0.874614197531
True Positives            1.0            1.0
True Negatives             -- 0.871541501976


(0.1253858024691358, 0.87461419753086422, 1.0, 0.87154150197628455)

In [59]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0256283068783 0.0239197530864
      Accuracy Rate  0.974371693122  0.976080246914
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.023919753086419752, 0.9760802469135802, 0.0, 1.0)

In [60]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0256283068783 0.0239197530864
 Accuracy Rate  0.974371693122  0.976080246914
True Positives             0.0             0.0
True Negatives              --             1.0


(0.023919753086419752, 0.9760802469135802, 0.0, 1.0)

In [61]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0289351851852 0.0239197530864
 Accuracy Rate  0.971064814815  0.976080246914
True Positives  0.438709677419  0.483870967742
True Negatives              --   0.98814229249


(0.023919753086419752,
 0.9760802469135802,
 0.4838709677419355,
 0.98814229249011853)

In [62]:
# define y
y = totalData.angry.values
y.shape

(8640,)

In [63]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [64]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [65]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.212632275132 0.212191358025
 Accuracy Rate 0.787367724868 0.787808641975
True Positives 0.281678773204 0.279383429672
True Negatives             -- 0.915098890497


(0.21219135802469136,
 0.78780864197530864,
 0.279383429672447,
 0.91509889049686444)

In [66]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.136243386243 0.130015432099
      Accuracy Rate 0.863756613757 0.869984567901
     True Positives 0.682808716707  0.66859344894
     True Negatives             -- 0.920405209841


(0.13001543209876543,
 0.86998456790123457,
 0.66859344894026973,
 0.92040520984081042)

In [67]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.210813492063 0.207175925926
 Accuracy Rate 0.789186507937 0.792824074074
True Positives 0.627118644068 0.597302504817
True Negatives             -- 0.841775205017


(0.20717592592592593,
 0.79282407407407407,
 0.59730250481695568,
 0.84177520501688374)

In [68]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set      Test Set   
-------------- -------------- ---------------
    Error Rate 0.195271164021  0.191358024691
 Accuracy Rate 0.804728835979  0.808641975309
True Positives 0.046811945117 0.0443159922929
True Negatives             --             1.0


(0.19135802469135801, 0.80864197530864201, 0.044315992292870907, 1.0)

In [69]:
# define y
y = totalData.hungry.values
y.shape

(8640,)

In [70]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [71]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [72]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00214947089947 0.00347222222222
 Accuracy Rate   0.997850529101   0.996527777778
True Positives   0.979651162791   0.961538461538
True Negatives               --    0.99837530463


(0.003472222222222222,
 0.99652777777777779,
 0.96153846153846156,
 0.99837530463038182)

In [73]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00380291005291 0.00501543209877
      Accuracy Rate   0.996197089947   0.994984567901
     True Positives   0.976744186047   0.953846153846
     True Negatives               --   0.997156783103


(0.0050154320987654318,
 0.99498456790123457,
 0.9538461538461539,
 0.99715678310316813)

In [74]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0562169312169 0.0478395061728
 Accuracy Rate  0.943783068783  0.952160493827
True Positives  0.982558139535  0.961538461538
True Negatives              --  0.951665312754


(0.047839506172839504,
 0.9521604938271605,
 0.96153846153846156,
 0.95166531275385868)

In [75]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00429894179894 0.00501543209877
 Accuracy Rate   0.995701058201   0.994984567901
True Positives   0.982558139535   0.961538461538
True Negatives               --   0.996750609261


(0.0050154320987654318,
 0.99498456790123457,
 0.96153846153846156,
 0.99675060926076364)

In [76]:
# define y
y = totalData.tired.values
y.shape

(8640,)

In [77]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [78]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [79]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0014880952381 0.00115740740741
 Accuracy Rate  0.998511904762   0.998842592593
True Positives             0.0              0.0
True Negatives              --              1.0


(0.0011574074074074073, 0.99884259259259256, 0.0, 1.0)

In [80]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set    
------------------- --------------- ----------------
         Error Rate 0.0014880952381 0.00115740740741
      Accuracy Rate  0.998511904762   0.998842592593
     True Positives             0.0              0.0
     True Negatives              --              1.0


(0.0011574074074074073, 0.99884259259259256, 0.0, 1.0)

In [81]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00380291005291 0.00578703703704
 Accuracy Rate   0.996197089947   0.994212962963
True Positives              0.0              0.0
True Negatives               --   0.995365005794


(0.0057870370370370367, 0.99421296296296291, 0.0, 0.99536500579374276)

In [82]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0014880952381 0.00115740740741
 Accuracy Rate  0.998511904762   0.998842592593
True Positives             0.0              0.0
True Negatives              --              1.0


(0.0011574074074074073, 0.99884259259259256, 0.0, 1.0)

In [83]:
# define y
y = totalData.bored.values
y.shape

(8640,)

In [84]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [85]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [86]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00082671957672 0.000385802469136
 Accuracy Rate   0.999173280423    0.999614197531
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00038580246913580245, 0.99961419753086422, 0.0, 1.0)

In [87]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set        Test Set    
------------------- ---------------- -----------------
         Error Rate 0.00082671957672 0.000385802469136
      Accuracy Rate   0.999173280423    0.999614197531
     True Positives              0.0               0.0
     True Negatives               --               1.0


(0.00038580246913580245, 0.99961419753086422, 0.0, 1.0)

In [88]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0209986772487 0.0204475308642
 Accuracy Rate  0.979001322751  0.979552469136
True Positives             0.0             0.0
True Negatives              --  0.979930528753


(0.020447530864197531, 0.97955246913580252, 0.0, 0.97993052875337705)

In [89]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00082671957672 0.000385802469136
 Accuracy Rate   0.999173280423    0.999614197531
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00038580246913580245, 0.99961419753086422, 0.0, 1.0)